In [ ]:
import panel as pn
import pydeck as pdk
pn.extension('deckgl')

In order to use Deck.gl you need a MAP BOX Key which you can acquire for free for limited use at [mapbox.com](https://account.mapbox.com/access-tokens/).

Now we can define a JSON spec and pass it to the DeckGL pane along with the Mapbox key:

## Advanced Example Based on Global Power Plant Database

We will base this example on data from the [Global Power Plant Database](http://datasets.wri.org/dataset/540dcf46-f287-47ac-985d-269b04bea4c6/resource/c240ed2e-1190-4d7e-b1da-c66b72e08858/download/globalpowerplantdatabasev120).

We will create an interactive application to explore the location and types of power plants globally.

In [ ]:
import param
import pandas as pd

POWER_PLANT_URL = (
    "https://raw.githubusercontent.com/MarcSkovMadsen/awesome-streamlit/master/"
    "gallery/global_power_plant_database/global_power_plant_database.csv"
)

MAPBOX_KEY = (
    "pk.eyJ1IjoicGFuZWxvcmciLCJhIjoiY2s1enA3ejhyMWhmZjNobjM1NXhtbWRrMyJ9.B_frQsAVepGIe-HiOJeqvQ"
)

pp_data = pd.read_csv(POWER_PLANT_URL)
pp_data.head(1)

Clean the data as PyDeck does not handle NA data well

In [ ]:
pp_data.primary_fuel = pp_data.primary_fuel.fillna("NA")
pp_data.capacity_mw = pp_data.capacity_mw.fillna(1)

Transform the data

In [ ]:
FUEL_COLORS = {
    "Oil": "black",
    "Solar": "green",
    "Gas": "black",
    "Other": "gray",
    "Hydro": "blue",
    "Coal": "black",
    "Petcoke": "black",
    "Biomass": "green",
    "Waste": "green",
    "Cogeneration": "gray",
    "Storage": "orange",
    "Wind": "green",
}

COLORS_R = {"black": 0, "green": 0, "blue": 0, "orange": 255, "gray": 128}
COLORS_G = {"black": 0, "green": 128, "blue": 0, "orange": 165, "gray": 128}
COLORS_B = {"black": 0, "green": 0, "blue": 255, "orange": 0, "gray": 128}

pp_data["primary_fuel_color"] = pp_data.primary_fuel.map(FUEL_COLORS)
pp_data["primary_fuel_color"] = pp_data["primary_fuel_color"].fillna("gray")
pp_data["color_r"] = pp_data["primary_fuel_color"].map(COLORS_R)
pp_data["color_g"] = pp_data["primary_fuel_color"].map(COLORS_G)
pp_data["color_b"] = pp_data["primary_fuel_color"].map(COLORS_B)
pp_data["color_a"] = 140

# "name", "primary_fuel", "capacity_mw",
pp_data = pp_data[[
    "latitude","longitude", "name", "capacity_mw",
    "color_r","color_g","color_b","color_a",]
]
pp_data.head(1)

In [ ]:
class GlobalPowerPlantDatabaseApp:
    data = param.DataFrame(pp_data)
    
    def _view_state(self):
        return pdk.ViewState(
            latitude=52.2323,
            longitude=-1.415,
            zoom=1,
            min_zoom=1,
            max_zoom=22,
            # pitch=40.5,
            # bearing=-27.36,
        )
    
    def _pp_layer(self):
        return pdk.Layer(
            "ScatterplotLayer",
            data=self.data,
            get_position=["longitude", "latitude"],
            get_fill_color="[color_r, color_g, color_b, color_a]",
            get_radius="capacity_mw*10",
            pickable=True,
            opacity=0.8,
            stroked=False,
            filled=True,
            wireframe=True,
        )
        
    def _deck(self):
        return pdk.Deck(
            map_style="mapbox://styles/mapbox/light-v9",
            initial_view_state=self._view_state(),
            layers=[self._pp_layer()],
            tooltip=True,
            mapbox_key=MAPBOX_KEY,
        )
    
    def view(self):
        return pn.pane.DeckGL(self._deck(), sizing_mode="stretch_width", height=700)

In [ ]:
app = GlobalPowerPlantDatabaseApp()
app.view()

### Credits

The DeckGl pane and this reference example was contributed by

- [Marc Skov Madsen](https://datamodelsanalytics.com)

and improved by

- Philipp Rüdiger